In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

In [66]:
df=pd.read_csv('D:/Python/Github_ML/Machine-Learning/data/_preprocessed.csv', index_col=0)
df.head()

,user-id,user-age,user-gender,user-nationality,user-knowledge,user-loyalty,user-loan,user-income,user-savings,user-properties,...,user-dependents,user-pension,product-type,product-risk,product-term,product-yield,transaction-id,year,month,score
0,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,1,1,30,1,5000,2019,9,0.061877
1,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,2,2,10,2,5001,2019,10,0.179746
2,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,0,2,10,2,5002,2019,9,0.767918
3,500,41,0,0,0,1,0,10822,132392,0,...,0,483040,0,0,20,0,5003,2017,12,0.606241
4,501,25,1,0,0,0,0,5284,51806,0,...,5,113300,0,0,1,0,5004,2019,10,0.829035


In [67]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale

In [68]:
#Поменяем индексы в датафрейме для удобства стандартизации
df=df.reindex(columns=[
 'user-age',
 'user-income',
 'user-savings',
 'user-properties',                   
 'user-dependents',  
 'user-pension',                   
 'product-term',   
 'year',
 'month',
    
 'user-gender',
 'user-nationality',
 'user-knowledge',
 'user-loyalty',
 'user-loan',
 'user-riskAversion',
 'user-marital',
 'product-type',
 'product-risk',
 'product-yield',
 'score'])

In [69]:
X = df.iloc[:, :-1].values
y = df["score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=4)

In [70]:
v=np.split(X_train, [9], axis=1)
w=np.split(X_test, [9], axis=1)

In [71]:
scaler = StandardScaler()
scaler.fit(v[0])


X_train=np.hstack((scaler.transform(v[0]), v[1]))

X_test=np.hstack((scaler.transform(w[0]), w[1]))

In [72]:
# Оценка
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def error_estimation(y_test, y_pred):
    mse = mean_squared_error(y_test, y_pred) 
    print("MSE: ",mse)
    
    print("RMSE: {}".format(np.sqrt(mse)))
    
    mae = mean_absolute_error(y_test, y_pred)
    print("MAE: ",mae)

    print("R2: ", r2_score(y_test, y_pred))

In [35]:
regressor = DecisionTreeRegressor()
regressor.fit(X_train, y_train)

DecisionTreeRegressor()

In [76]:
error_estimation(y_test, regressor.predict(X_test)) #Результат обычного обучения (провал)

MSE:  0.16957258851371118
RMSE: 0.41179192380826407
MAE:  0.3285299254627722
R2:  -1.0510679682237725


In [77]:
from sklearn.ensemble import RandomForestRegressor

In [78]:
clf=RandomForestRegressor(n_estimators=20)
clf.fit(X_train, y_train)
error_estimation(y_test, clf.predict(X_test)) #Результат обучения Rand.For/ (провал)

MSE:  0.09259329620234637
RMSE: 0.30429146587169736
MAE:  0.2552797338894647
R2:  -0.11996370154798042


## Bagging

In [45]:
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor

In [79]:
bg=BaggingRegressor(DecisionTreeRegressor(), max_samples=0.5, max_features=1.0, n_estimators=20) 
bg.fit(X_train,y_train)
error_estimation(y_test, bg.predict(X_test))

MSE:  0.08916336062008248
RMSE: 0.29860234530238117
MAE:  0.2520338678610941
R2:  -0.07847686061741577


## Boosting 

In [80]:
from sklearn.ensemble import AdaBoostRegressor

In [87]:
adb=AdaBoostRegressor(DecisionTreeRegressor(), n_estimators=20, learning_rate=1)
adb.fit(X_train,y_train)

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), learning_rate=1,
                  n_estimators=20)

In [88]:
error_estimation(y_test, adb.predict(X_test))
y_pred = adb.predict(X_test)

MSE:  0.10951344288114542
RMSE: 0.33092815365445327
MAE:  0.26240016773461355
R2:  -0.32462160749087676


In [89]:
[print(f'{elem[0]}||||{elem[1]}') for elem in zip(y_test, y_pred)];

0.637008056618801||||0.5830078308779097
0.7473520592931727||||0.5981219894863372
0.2252716355621957||||0.2600909454321547
0.6682161864214831||||0.5867468083110134
0.8392620398114301||||0.6208334636318229
0.1664352243798691||||0.5812430961368535
0.1302577986527882||||0.5416588361590855
0.2432643522098824||||0.7125355543202708
0.5208024675481202||||0.674412831918678
0.9432322525028436||||0.5812430961368535
0.6909798890972237||||0.5884821147399817
0.9949830292601434||||0.6198169808612012
0.2672416082418039||||0.5637589292302011
0.6893726197163159||||0.6649151817216962
0.7823147672885388||||0.2995023685425926
0.8992496897972042||||0.6786816990017145
0.888491371526923||||0.568755632668859
0.0||||0.2811168574602318
0.0||||0.6225519560922249
0.5172703924069402||||0.5102779555568386
0.0410362432075503||||0.6347624475699141
0.5736437380398917||||0.5032405620903629
0.2251650961420958||||0.6869454328843594
0.5488309555276203||||0.5815718843754604
1.0||||0.7202125343625008
0.5211672109184555||||0.

## Stacking

In [133]:
from sklearn.ensemble import StackingRegressor

In [134]:
estimators = [
    ('rf', RandomForestRegressor(n_estimators=20, random_state=42)),
    ('svr', make_pipeline(StandardScaler(), LinearRegression()))]

regressor = StackingRegressor(estimators = estimators)
regressor.fit(X_train, y_train)

StackingRegressor(estimators=[('rf',
                               RandomForestRegressor(n_estimators=20,
                                                     random_state=42)),
                              ('svr',
                               Pipeline(steps=[('standardscaler',
                                                StandardScaler()),
                                               ('linearregression',
                                                LinearRegression())]))])

In [138]:
error_estimation(y_train, regressor.predict(X_train))

MSE:  0.08189384018872686
RMSE: 0.2861709981614609
MAE:  0.23985189829471687
R2:  0.017785450431674343


# Task 2 

In [114]:
!pip install catboost

In [120]:
from catboost import * 


In [129]:
from catboost import CatBoostRegressor

model = CatBoostRegressor(
    task_type = "CPU",
    learning_rate = 0.1,
    use_best_model = True,
    loss_function = "RMSE"
)


model.fit(X_train, y_train,
         eval_set = (X_test, y_test),
         verbose = False,
         plot = True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [130]:
error_estimation(y_test, model.predict(X_test))

MSE:  0.08255283471151102
RMSE: 0.2873200910335214
MAE:  0.23988093132958896
R2:  0.0014808616950436893
